In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css: string = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# High-Performance Set Theory Engine

This module implements a ZFC-inspired Set and Map engine focused on **Strict Immutability** and **Structural Equality**. Unlike standard JavaScript reference comparisons, this engine treats data structures as mathematical values: two sets are equal if and only if they contain the same elements, regardless of their memory address.

## Core Constraints & Contract
To achieve raw performance, we enforce a strict contract:
1.  **Finite Universe:** Only finite numbers are supported. `NaN` and `Infinity` are illegal to ensure strict ordering logic.
2.  **Immutability:** Once created, containers (`RecursiveSet`, `Tuple`, `RecursiveMap`) must not be mutated.
3.  **Type Consistency:** Hash collisions between distinct types (e.g., an Array and a Tuple with identical content) are resolved via type scoring, but users should avoid mixing structural types for the same logical data.

## Forward Declarations (Class Stubs)

TypeScript's strict type system cannot easily resolve circular dependencies where a type definition relies on a class, and that class relies on the type definition (e.g., `RecursiveSet<T>` uses `Value`, but `Value` includes `RecursiveSet`).

To resolve this **Henne-Ei-Problem** (chicken-and-egg problem) within the Jupyter compilation context, we declare generic class stubs first. These stubs define the structural shape required by the `Value` type (specifically the `hashCode` property) without imposing the final constraints immediately.

In [ ]:
class RecursiveSet<T> {
    readonly hashCode!: number;
    compare(o: object): number { return 0; }
}

class Tuple<T> {
    readonly hashCode!: number;
    readonly raw!: ReadonlyArray<T>;
}

class RecursiveMap<K, V> {
    readonly hashCode!: number;
    compare(o: object): number { return 0; }
}

## Recursive Type Definitions

We define the universe of discourse `Value` as a recursive union type. This ensures the engine is closed under its own operations (a Set can contain other Sets).

### The Value Type
A `Value` is defined as:
$$V = P \cup S \cup T \cup M \cup A$$
Where:
* $P$: Primitives (Numbers, Strings)
* $S$: RecursiveSet containing Values
* $T$: Tuple containing Values
* $M$: RecursiveMap mapping Values to Values
* $A$: ReadonlyArray of Values

**Note:** We strictly define `RecursiveMap` as `<Value, Value>` to ensure both keys and values are mathematically comparable within our system.

In [ ]:
type Primitive = number | string;

type Value =
    | Primitive
    | RecursiveSet<Value>
    | Tuple<Value>
    | RecursiveMap<Value, Value>
    | ReadonlyArray<Value>;

## Fast Hashing (FNV-1a)

Structural equality requires $O(1)$ equality checks, which we achieve via pre-computed hashes. We utilize the **[Fowler–Noll–Vo (FNV-1a)](https://en.wikipedia.org/wiki/Fowler%E2%80%93Noll%E2%80%93Vo_hash_function)** algorithm. It is non-cryptographic but chosen for its high dispersion and low computational overhead, making it ideal for hashing small-to-medium keys in hash maps and sets.

### Mathematical Definition
The hash $h$ is calculated iteratively for each byte $d$ of the data. We operate in the 32-bit universe modulo $2^{32}$.

Given:
* **FNV Prime** $p = 16777619$
* **Offset Basis** $h_{basis} = 2166136261$

The algorithm is defined recursively:
$$
\begin{aligned}
h_0 &= h_{basis} \\
h_{i+1} &= (h_i \oplus d_i) \cdot p \pmod{2^{32}}
\end{aligned}
$$
where $\oplus$ denotes the bitwise XOR operation.

### Implementation Optimizations
To optimize for the V8 JavaScript engine:
1.  **Integer Fast-Path:** If a number is a safe 32-bit integer, we bypass the hashing algorithm and return the integer itself. This provides a massive speedup for array indices.
2.  **Float64 Buffering:** Non-integer numbers are written to a static `ArrayBuffer` as 64-bit floats (IEEE 754). We hash the lower and upper 32-bit words separately using a reusable `DataView` to prevent garbage collection overhead.

In [ ]:
const FNV_PRIME = 16777619;
const FNV_OFFSET = 2166136261;
const floatBuffer = new ArrayBuffer(8);
const view = new DataView(floatBuffer);

function hashNumber(val: number): number {
    if ((val | 0) === val) return val | 0;

    view.setFloat64(0, val, true);
    let h = FNV_OFFSET;
    h ^= view.getInt32(0, true);
    h = Math.imul(h, FNV_PRIME);
    h ^= view.getInt32(4, true);
    h = Math.imul(h, FNV_PRIME);
    return h >>> 0;
}

### String Hashing (FNV-1a Implementation)

Strings are hashed by iterating over their UTF-16 code units. This process runs in linear time $O(L)$, where $L$ is the string length.

**Critical Implementation Details:**
1.  **32-Bit Multiplication (`Math.imul`):** Standard JavaScript multiplication `*` operates on floating-point numbers (IEEE 754), which loses precision for large integers beyond $2^{53}$. To simulate standard CPU 32-bit integer multiplication correctly (allowing overflow to wrap around modulo $2^{32}$), we **must** use `Math.imul`.
2.  **Unsigned Cast (`>>> 0`):** Bitwise operators in JavaScript (like `^` or `|`) return signed 32-bit integers (Two's complement). The final `>>> 0` (zero-fill right shift) forces the runtime to interpret the bit pattern as an **unsigned** 32-bit integer.

$$h_{final} \in [0, 2^{32}-1]$$

In [ ]:
function hashString(str: string): number {
    let h = FNV_OFFSET;
    const len = str.length;
    for (let i = 0; i < len; i++) {
        h ^= str.charCodeAt(i);
        h = Math.imul(h, FNV_PRIME);
    }
    return h >>> 0;
}

### Universal Recursive Dispatcher

The `hashValue` function is the engine's entry point for structural hashing. It acts as a **polymorphic dispatcher** that determines the hashing strategy based on the runtime type of the value.

**Strategy Hierarchy:**
1.  **Primitives:** Delegates immediately to specialized functions (`hashNumber`, `hashString`).
2.  **Cached Containers ($O(1)$):** If the value is a `RecursiveSet`, `Tuple`, or `RecursiveMap`, we read its `.hashCode` property. This is a critical optimization: these immutable structures compute their hash **once** (memoization) and freeze. We never re-traverse deep structures.
3.  **Arrays ($O(N)$):** Arrays are treated as value types. We iterate recursively:
    $$Hash([v_0, v_1, \dots]) = FNV(Hash(v_0), FNV(Hash(v_1), \dots))$$
    This ensures that order is significant: $Hash([a, b]) \neq Hash([b, a])$.

In [ ]:
function hashValue(v: Value): number {
    if (typeof v === 'number') return hashNumber(v);
    if (typeof v === 'string') return hashString(v);

    if (v instanceof RecursiveSet || v instanceof Tuple || v instanceof RecursiveMap) {
        return v.hashCode;
    }

    if (Array.isArray(v)) {
        let h = FNV_OFFSET;
        for (let i = 0; i < v.length; i++) {
            h ^= hashValue(v[i]);
            h = Math.imul(h, FNV_PRIME);
        }
        return h >>> 0;
    }

    return 0;
}

## Structural Comparison Logic

To establish a strict **Total Ordering** over the entire universe of values, we first need a method to compare sequences (Arrays and Tuples).

### Global Comparator (Total Ordering)

The `compare` function is the heart of the engine, providing a deterministic sorting order for **any** two values in the universe. This allows us to sort mixed sets (e.g., `{1, "A", [2]}`) consistently.

### Comparison Pipeline
The logic follows a strict hierarchy of checks to maximize speed:

1.  **Identity ($O(1)$):** If $a \equiv b$, return 0 immediately.
2.  **Type Separation:** We impose an artificial ordering on types to avoid comparing incompatible structures (e.g., comparing a Number to a String).
    $$\text{TypeOrder}: \text{Number} < \text{String} < \text{Complex Types}$$
3.  **Hash Short-Circuit ($O(1)$):** Before expensive deep traversals, we check the hashes.
    $$\text{if } h(a) \neq h(b) \implies \text{return } h(a) - h(b)$$
    This guarantees that structurally distinct objects are separated quickly.
4.  **Deep Structural Comparison:** If hashes collide (or are equal), we dispatch to the specific comparison logic for that type (`Set.compare`, `Map.compare`, or `compareSequences`).

**Safety Contract:**
* **Finite Numbers Only:** We use `a - b` for number comparison. `NaN` or `Infinity` will break sorting invariants.
* **Type Consistency:** Users must avoid mixing different structural types (e.g., Array vs. Tuple) that effectively represent the same data, as hash collisions between them rely solely on arbitrary type implementation details.

### Lexicographical Sequence Comparison
Sequences are compared lexicographically, similar to words in a dictionary.
Given two sequences $A = [a_0, a_1, \dots, a_m]$ and $B = [b_0, b_1, \dots, b_n]$, the comparison $C(A, B)$ is defined as:

1.  **Length Check:** If $|A| \neq |B|$, the shorter sequence comes first. This is an optimization for performance (length is $O(1)$) and acts as a simple discriminator.
    $$\text{if } |A| \neq |B| \implies \text{return } |A| - |B|$$
2.  **Element-wise Scan:** If lengths are equal, we iterate from $i = 0$ to $n$:
    * Compare elements $a_i$ and $b_i$.
    * If $a_i \neq b_i$, the result is strictly determined by their difference.
    * If all elements are equal, the sequences are equal.

$$C(A, B) = \begin{cases} C(a_i, b_i) & \text{if } \exists i: a_i \neq b_i \text{ (first mismatch)} \\ 0 & \text{otherwise} \end{cases}$$

In [ ]:
function compare(a: Value, b: Value): number {
    if (a === b) return 0;

    if (typeof a === 'number' && typeof b === 'number') return a - b;
    if (typeof a === 'string' && typeof b === 'string') return a < b ? -1 : 1;

    const typeA = typeof a;
    const typeB = typeof b;

    if (typeA !== typeB) {
        const scoreA = (typeA === 'number') ? 1 : (typeA === 'string' ? 2 : 3);
        const scoreB = (typeB === 'number') ? 1 : (typeB === 'string' ? 2 : 3);
        return scoreA - scoreB;
    }

    const h1 = hashValue(a);
    const h2 = hashValue(b);
    if (h1 !== h2) return h1 < h2 ? -1 : 1;

    if (a instanceof RecursiveSet && b instanceof RecursiveSet) return a.compare(b);
    if (a instanceof RecursiveMap && b instanceof RecursiveMap) return a.compare(b);
    if (a instanceof Tuple && b instanceof Tuple) return compareSequences(a.raw, b.raw);
    if (Array.isArray(a) && Array.isArray(b)) return compareSequences(a, b);

    return 0;
}

function compareSequences(a: ReadonlyArray<Value>, b: ReadonlyArray<Value>): number {
    const len = a.length;
    if (len !== b.length) return len - b.length;

    for (let i = 0; i < len; i++) {
        const diff = compare(a[i], b[i]);
        if (diff !== 0) return diff;
    }
    return 0;
}

## Immutable Tuple Container

The `Tuple` class provides a fixed-size, ordered sequence of elements. Unlike standard JavaScript Arrays, Tuples in this library are strictly **immutable** and treated as value objects. This means two tuples are considered equal if their contents are identical, regardless of their reference identity.

### Contract & Invariants
1.  **Defensive Copying:** Upon construction, the input arguments are shallow-copied via `.slice()`. This isolates the Tuple from external mutation of the source array.
2.  **Immutability:** The internal storage is frozen using `Object.freeze()`. This prevents the addition, removal, or replacement of elements.
3.  **Hash Caching:** Since the content is constant, we calculate the FNV-1a hash code **once** during construction ($O(N)$) and cache it ($O(1)$ access). This makes Tuples ideal keys for HashMaps.

### Mathematical Definition
A Tuple $T$ is a finite ordered list:
$$T = (v_1, v_2, \dots, v_n)$$

Equality is defined structurally:
$$A = B \iff (|A| = |B|) \land (\forall i \in [0, |A|-1]: A_i = B_i)$$

> **Note:** The immutability is shallow. If a Tuple contains a mutable object (like a standard Array), that object can still be modified. For strict safety, only store `Value` types (Primitives, RecursiveSets, Tuples).

In [ ]:
class Tuple<T extends Value[]> {
    readonly #values: ReadonlyArray<Value>;
    readonly hashCode: number;

    constructor(...values: T) {
        this.#values = values.slice();
        Object.freeze(this.#values);
        this.hashCode = hashValue(this.#values);
    }

    get raw(): ReadonlyArray<Value> { return this.#values; }
    get length(): number { return this.#values.length; }
    get values(): ReadonlyArray<Value> { return this.#values; }

    *[Symbol.iterator](): Iterator<Value> { yield* this.#values; }

    toString(): string { return `(${this.#values.join(', ')})`; }

    [Symbol.for('nodejs.util.inspect.custom')]() { return this.toString(); }
}

## Set Theory Algorithms (The Engine)

Before defining the container class, we implement the core set-theoretic operations as pure functions operating on sorted arrays. This separation of concerns simplifies the complexity and allows us to focus on the **Merge Scan** algorithms.

All operations assume the input arrays $A$ and $B$ are strictly **sorted** according to our total ordering.

### 1. Deduplication (Unique)
To guarantee the set invariant $\forall x, y \in S: x \neq y$, we utilize a linear scan $O(N)$ on the sorted array to filter duplicates.

### 2. Union ($A \cup B$)
We perform a **Merge Scan** (similar to Merge Sort) to combine two sorted arrays into one.
* **Complexity:** $O(|A| + |B|)$
* **Logic:** We iterate through both arrays simultaneously. If $a < b$, we take $a$. If $b < a$, we take $b$. If $a = b$, we take one and advance both pointers.

### 3. Intersection ($A \cap B$)
A synchronous scan that retains elements only if they appear in both sequences.
* **Complexity:** $O(|A| + |B|)$

### 4. Difference ($A \setminus B$)
Retains elements from $A$ only if they do not appear in $B$.

In [ ]:
// ============================================================================
// PURE ALGORITHMS (Operating on Sorted ReadonlyArrays)
// ============================================================================

function computeUnique<T extends Value>(sorted: T[]): T[] {
    if (sorted.length < 2) return sorted;
    const out: T[] = [sorted[0]];
    let last = sorted[0];
    const len = sorted.length;
    for (let i = 1; i < len; i++) {
        const curr = sorted[i];
        if (compare(curr, last) !== 0) {
            out.push(curr);
            last = curr;
        }
    }
    return out;
}

function computeUnion<T extends Value, U extends Value>(A: ReadonlyArray<T>, B: ReadonlyArray<U>): (T | U)[] {
    const res: (T | U)[] = [];
    let i = 0, j = 0;
    const lenA = A.length, lenB = B.length;

    while (i < lenA && j < lenB) {
        const cmp = compare(A[i], B[j]);
        if (cmp < 0) res.push(A[i++]);
        else if (cmp > 0) res.push(B[j++]);
        else { res.push(A[i++]); j++; }
    }
    while (i < lenA) res.push(A[i++]);
    while (j < lenB) res.push(B[j++]);
    return res;
}

function computeIntersection<T extends Value>(A: ReadonlyArray<T>, B: ReadonlyArray<T>): T[] {
    const res: T[] = [];
    let i = 0, j = 0;
    const lenA = A.length, lenB = B.length;

    while (i < lenA && j < lenB) {
        const cmp = compare(A[i], B[j]);
        if (cmp < 0) i++;
        else if (cmp > 0) j++;
        else { res.push(A[i++]); j++; }
    }
    return res;
}

function computeDifference<T extends Value>(A: ReadonlyArray<T>, B: ReadonlyArray<T>): T[] {
    const res: T[] = [];
    let i = 0, j = 0;
    const lenA = A.length, lenB = B.length;

    while (i < lenA && j < lenB) {
        const cmp = compare(A[i], B[j]);
        if (cmp < 0) res.push(A[i++]);
        else if (cmp > 0) j++;
        else { i++; j++; }
    }
    while (i < lenA) res.push(A[i++]);
    return res;
}

function computeSymmetricDifference<T extends Value>(A: ReadonlyArray<T>, B: ReadonlyArray<T>): T[] {
    const res: T[] = [];
    let i = 0, j = 0;
    const lenA = A.length, lenB = B.length;

    while (i < lenA && j < lenB) {
        const cmp = compare(A[i], B[j]);
        if (cmp < 0) res.push(A[i++]);
        else if (cmp > 0) res.push(B[j++]);
        else { i++; j++; }
    }
    while (i < lenA) res.push(A[i++]);
    while (j < lenB) res.push(B[j++]);
    return res;
}

## RecursiveSet Implementation

With the complex algorithms abstracted away, we can now implement the `RecursiveSet` class. This class acts as a high-performance wrapper around a sorted array, enforcing immutability and state management.

### Lifecycle & Safety
1.  **Sorted Storage:** Elements are always stored sorted (`#elements`). This enables binary search and the fast merge operations defined above.
2.  **Freeze-on-Hash:** The set is effectively immutable. However, to allow efficient construction, we allow mutation (add/remove) *until* the `hashCode` is accessed. Once hashed, the set sets `#isFrozen = true`, and any subsequent mutation attempt throws an error.
3.  **Factories:**
    * `constructor`: Sorts and deduplicates ($O(N \log N)$).
    * `fromSortedUnsafe`: Trusted creation ($O(1)$) used by internal methods.

In [ ]:
class RecursiveSet<T extends Value> {
    #elements: T[];
    #hashCode: number | null = null;
    #isFrozen: boolean = false;

    static compare(a: unknown, b: unknown): number { return compare(a as Value, b as Value); }

    constructor(...elements: T[]) {
        if (elements.length > 1) {
            elements.sort(compare);
            this.#elements = computeUnique(elements);
        } else {
            this.#elements = elements;
        }
    }

    static fromArray<T extends Value>(elements: T[]): RecursiveSet<T> {
        const s = new RecursiveSet<T>();
        if (elements.length > 1) {
            elements.sort(compare);
            s.#elements = computeUnique(elements);
        } else {
            s.#elements = elements;
        }
        return s;
    }

    static fromSortedUnsafe<T extends Value>(sortedUnique: T[]): RecursiveSet<T> {
        const s = new RecursiveSet<T>();
        s.#elements = sortedUnique;
        return s;
    }

    #checkFrozen(op: string) {
        if (this.#isFrozen) {
            throw new Error(`InvalidOperation: Cannot ${op} a frozen RecursiveSet. Use mutableCopy().`);
        }
    }

    get raw(): readonly T[] { return this.#elements; }
    get isFrozen(): boolean { return this.#isFrozen; }
    get size(): number { return this.#elements.length; }
    isEmpty(): boolean { return this.#elements.length === 0; }

    get hashCode(): number {
        if (this.#hashCode !== null) return this.#hashCode;
        let h = 0;
        const arr = this.#elements;
        const len = arr.length;
        for (let i = 0; i < len; i++) {
            h = (Math.imul(31, h) + hashValue(arr[i])) | 0;
        }
        this.#hashCode = h;
        this.#isFrozen = true;
        return h;
    }

    compare(other: RecursiveSet<Value>): number {
        if (this === other) return 0;
        const h1 = this.hashCode;
        const h2 = other.hashCode;
        if (h1 !== h2) return h1 < h2 ? -1 : 1;
        
        // Deep compare using the global comparator on arrays
        return compareSequences(this.#elements, other.#elements);
    }

    equals(other: RecursiveSet<Value>): boolean { return this.compare(other) === 0; }

    // === DELEGATED SET OPERATIONS ===

    union<U extends Value>(other: RecursiveSet<U>): RecursiveSet<T | U> {
        const merged = computeUnion(this.#elements, other.raw);
        return RecursiveSet.fromSortedUnsafe(merged as any);
    }

    intersection(other: RecursiveSet<T>): RecursiveSet<T> {
        const merged = computeIntersection(this.#elements, other.raw);
        return RecursiveSet.fromSortedUnsafe(merged);
    }

    difference(other: RecursiveSet<T>): RecursiveSet<T> {
        const merged = computeDifference(this.#elements, other.raw);
        return RecursiveSet.fromSortedUnsafe(merged);
    }

    symmetricDifference(other: RecursiveSet<T>): RecursiveSet<T> {
        const merged = computeSymmetricDifference(this.#elements, other.raw);
        return RecursiveSet.fromSortedUnsafe(merged);
    }

    cartesianProduct<U extends Value>(other: RecursiveSet<U>): RecursiveSet<Tuple<[T, U]>> {
        const result: Tuple<[T, U]>[] = [];
        const arrA = this.#elements;
        const arrB = other.raw;
        const lenA = arrA.length;
        const lenB = arrB.length;

        for (let i = 0; i < lenA; i++) {
            const a = arrA[i];
            for (let j = 0; j < lenB; j++) {
                result.push(new Tuple(a, arrB[j]));
            }
        }
        result.sort(compare);
        return RecursiveSet.fromSortedUnsafe(result);
    }

    powerset(): RecursiveSet<RecursiveSet<T>> {
        const n = this.size;
        if (n > 20) throw new Error("Powerset too large");
        
        const subsets: RecursiveSet<T>[] = [];
        const max = 1 << n;
        for (let i = 0; i < max; i++) {
            const subsetElements: T[] = [];
            for (let j = 0; j < n; j++) {
                if (i & (1 << j)) subsetElements.push(this.#elements[j]);
            }
            subsets.push(RecursiveSet.fromSortedUnsafe(subsetElements));
        }
        return RecursiveSet.fromArray(subsets);
    }

    // === MUTATION & UTILS ===

    has(element: T): boolean {
        const arr = this.#elements;
        const len = arr.length;
        if (len < 10) {
            for (let i = 0; i < len; i++) {
                if (compare(arr[i], element) === 0) return true;
            }
            return false;
        }
        let low = 0, high = len - 1;
        while (low <= high) {
            const mid = (low + high) >>> 1;
            const cmp = compare(arr[mid], element);
            if (cmp === 0) return true;
            if (cmp < 0) low = mid + 1;
            else high = mid - 1;
        }
        return false;
    }

    add(element: T): this {
        this.#checkFrozen('add() to');
        const arr = this.#elements;
        if (arr.length > 0) {
            const lastCmp = compare(arr[arr.length-1], element);
            if (lastCmp < 0) {
                arr.push(element);
                this.#hashCode = null;
                return this;
            }
            if (lastCmp === 0) return this;
        }

        let low = 0, high = arr.length - 1, idx = arr.length;
        while (low <= high) {
            const mid = (low + high) >>> 1;
            const cmp = compare(arr[mid], element);
            if (cmp === 0) return this;
            if (cmp < 0) low = mid + 1;
            else { idx = mid; high = mid - 1; }
        }
        arr.splice(idx, 0, element);
        this.#hashCode = null;
        return this;
    }

    remove(element: T): this {
        this.#checkFrozen('remove() from');
        const arr = this.#elements;
        let low = 0, high = arr.length - 1;
        while (low <= high) {
            const mid = (low + high) >>> 1;
            const cmp = compare(arr[mid], element);
            if (cmp === 0) {
                arr.splice(mid, 1);
                this.#hashCode = null;
                return this;
            }
            if (cmp < 0) low = mid + 1;
            else high = mid - 1;
        }
        return this;
    }

    clear(): this {
        this.#checkFrozen('clear()');
        this.#elements = [];
        this.#hashCode = 0;
        return this;
    }

    mutableCopy(): RecursiveSet<T> {
        const s = new RecursiveSet<T>();
        s.#elements = this.#elements.slice();
        return s;
    }
    
    clone(): RecursiveSet<T> { return this.mutableCopy(); }

    pickRandom(): T {
        const arr = this.#elements;
        const idx = (Math.random() * arr.length) | 0;
        return arr[idx]!;
    }

    isSubset(other: RecursiveSet<T>): boolean {
        if (this.size > other.size) return false;
        let i = 0, j = 0;
        const A = this.#elements, B = other.raw;
        while (i < A.length && j < B.length) {
            const cmp = compare(A[i], B[j]);
            if (cmp < 0) return false;
            if (cmp > 0) j++;
            else { i++; j++; }
        }
        return i === A.length;
    }

    isSuperset(other: RecursiveSet<T>): boolean { return other.isSubset(this); }
    
    *[Symbol.iterator](): Iterator<T> { yield* this.#elements; }
    
    toString(): string { return `{${this.#elements.join(', ')}}`; }
    
    [Symbol.for('nodejs.util.inspect.custom')]() { return this.toString(); }
}

// Factories
function emptySet<T extends Value>(): RecursiveSet<T> { return new RecursiveSet<T>(); }
function singleton<T extends Value>(element: T): RecursiveSet<T> { return new RecursiveSet<T>(element); }

## Recursive Map Implementation

The `RecursiveMap<K, V>` is an immutable dictionary that maps Keys to Values.

### Architectural Decisions

#### 1. Deep Value Equality for Keys
Standard JavaScript `Map`s use **Reference Equality** for objects. This means `map.get([1, 2])` returns `undefined` even if you previously set `map.set([1, 2], "value")`, because the two arrays are different references.

Our `RecursiveMap` uses **Structural Equality**.
$$Map([1, 2]) \equiv Map([1, 2])$$
This allows using complex objects (Tuples, Sets) as keys effectively.

#### 2. Storage: Sorted Array of Entries
Instead of a Hash Table (which is hard to make immutable and memory-efficient), we store entries as a sorted array of `{key, value}` objects.
* **Storage:** `[{key: A, val: 1}, {key: B, val: 2}]` sorted by $K$.
* **Lookup:** We perform a **Binary Search** on the keys.
    * Complexity: $O(\log N)$
* **Memory:** extremely compact compared to hash buckets.

#### 3. Hashing
The hash code of the map is a deterministic combination of all key-value pairs.
$$H(Map) = \bigoplus (Hash(k_i) \cdot 31 \oplus Hash(v_i))$$
This ensures that if two maps contain the same data, they have the same hash.

In [ ]:
class RecursiveMap<K extends Value, V extends Value> {
    #entries: Array<{ key: K, value: V }>;
    #hashCode: number | null = null;
    #isFrozen: boolean = false;

    constructor(entries?: Iterable<[K, V]>) {
        this.#entries = [];
        if (entries) {
            for (const [k, v] of entries) {
                this.set(k, v);
            }
        }
    }

    // === CRITICAL INFRASTRUCTURE ===

    #checkFrozen(op: string) {
        if (this.#isFrozen) {
            throw new Error(`InvalidOperation: Cannot ${op} a frozen RecursiveMap. Use mutableCopy().`);
        }
    }

    get size(): number { return this.#entries.length; }

    isEmpty(): boolean { return this.#entries.length === 0; }

    get hashCode(): number {
        if (this.#hashCode !== null) return this.#hashCode;
        let h = 0;
        const len = this.#entries.length;
        // Map Hash: Combine Hash(Key) and Hash(Value) using XOR mixing
        for (let i = 0; i < len; i++) {
            const entry = this.#entries[i];
            const hKey = hashValue(entry.key);
            const hVal = hashValue(entry.value);
            // Mix key and value hashes
            const entryHash = (Math.imul(hKey, 31) ^ hVal) | 0;
            h = (Math.imul(31, h) + entryHash) | 0;
        }
        this.#hashCode = h;
        this.#isFrozen = true;
        return h;
    }

    /**
     * Deep comparison of two maps.
     * Maps are equal if they have the same size and identical (key, value) pairs.
     */
    compare(other: RecursiveMap<Value, Value>): number {
        if (this === other) return 0;

        const lenA = this.#entries.length;
        const lenB = other.#entries.length;
        if (lenA !== lenB) return lenA - lenB;

        // Since entries are sorted by Key, we can iterate linearly (Synchronous Scan).
        for (let i = 0; i < lenA; i++) {
            const entryA = this.#entries[i];
            const entryB = other.#entries[i];

            // 1. Compare Keys
            const cmpKey = compare(entryA.key, entryB.key);
            if (cmpKey !== 0) return cmpKey;

            // 2. If Keys are equal, compare Values
            const cmpVal = compare(entryA.value, entryB.value);
            if (cmpVal !== 0) return cmpVal;
        }
        return 0;
    }

    equals(other: RecursiveMap<Value, Value>): boolean { return this.compare(other) === 0; }

    // === DATA ACCESS (Binary Search) ===

    /**
     * Binary Search for Key Index.
     * @returns index if found (>= 0), or bitwise complement (~index) of insertion point if not found.
     */
    #indexOf(key: K): number {
        const arr = this.#entries;
        let low = 0, high = arr.length - 1;

        while (low <= high) {
            const mid = (low + high) >>> 1;
            const cmp = compare(arr[mid].key, key);
            if (cmp === 0) return mid;
            if (cmp < 0) low = mid + 1;
            else high = mid - 1;
        }
        return ~low; // Returns - (insertionPoint + 1)
    }

    has(key: K): boolean {
        return this.#indexOf(key) >= 0;
    }

    get(key: K): V | undefined {
        const idx = this.#indexOf(key);
        return idx >= 0 ? this.#entries[idx].value : undefined;
    }

    // === MUTATION ===

    set(key: K, value: V): this {
        this.#checkFrozen('set() on');
        const idx = this.#indexOf(key);

        if (idx >= 0) {
            // Update existing key
            // Optimization: If value is structurally equal, do nothing to preserve hash/immutability
            if (compare(this.#entries[idx].value, value) !== 0) {
                this.#entries[idx].value = value;
                this.#hashCode = null;
            }
        } else {
            // Insert new key at sorted position
            const insertPos = ~idx;
            this.#entries.splice(insertPos, 0, { key, value });
            this.#hashCode = null;
        }
        return this;
    }

    delete(key: K): boolean {
        this.#checkFrozen('delete() from');
        const idx = this.#indexOf(key);
        if (idx >= 0) {
            this.#entries.splice(idx, 1);
            this.#hashCode = null;
            return true;
        }
        return false;
    }

    clear(): this {
        this.#checkFrozen('clear()');
        this.#entries = [];
        this.#hashCode = null;
        return this;
    }

    mutableCopy(): RecursiveMap<K, V> {
        const map = new RecursiveMap<K, V>();
        // Shallow copy the array of objects.
        // Since K and V are immutable Value types, this is safe.
        map.#entries = this.#entries.map(e => ({ key: e.key, value: e.value }));
        return map;
    }

    clone(): RecursiveMap<K, V> { return this.mutableCopy(); }

    // === ITERATORS & UTILS ===

    keys(): K[] { return this.#entries.map(e => e.key); }
    values(): V[] { return this.#entries.map(e => e.value); }
    entries(): [K, V][] { return this.#entries.map(e => [e.key, e.value]); }

    *[Symbol.iterator](): Iterator<[K, V]> {
        for (const e of this.#entries) {
            yield [e.key, e.value];
        }
    }

    toString(): string {
        const body = this.#entries
            .map(e => `${String(e.key)} => ${String(e.value)}`)
            .join(', ');
        return `Map{${body}}`;
    }

    [Symbol.for('nodejs.util.inspect.custom')]() { return this.toString(); }
}

## Factory Functions & Utilities

To simplify the API and support functional programming patterns, we provide factory functions. These helper methods abstract away the `new` keyword and allow for cleaner composition of set operations.

* `emptySet()`: Returns a typed, empty `RecursiveSet`.
* `singleton(x)`: Creates a set containing exactly one element `{x}`.

These are particularly useful for recursive algorithms (like automata construction) where base cases often involve empty sets or singletons.

In [ ]:
function emptySet<T extends Value>(): RecursiveSet<T> { return new RecursiveSet<T>(); }
function singleton<T extends Value>(element: T): RecursiveSet<T> { return new RecursiveSet<T>(element); }